In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import nibabel as nib
import scipy as sp
from sklearn.linear_model import LinearRegression
import sys
from pathlib import Path
import utils
import matplotlib.pyplot as plt

In [ ]:
participant_data = pd.read_csv('/redacted/path/demographics.csv')
age = np.array(participant_data.age)
sex = np.array(participant_data.sex)
participantid = np.array(participant_data.ID)

In [ ]:
metrics = ['fneurite','fsoma','fe','R','ODI','wic','b500_1200_MD','gyrification','thickness','curvature','AP','PD','IO']
hemi = ['L','R']
metrichold = np.empty((7262,2,len(participantid),len(metrics)))

### Load surface maps for all measures

In [ ]:
for ii in range(len(participantid)):
    for jj in range(len(hemi)):
        for kk in range(len(metrics)):
            if metrics[kk] == 'gyrification' or metrics[kk] == 'thickness' or metrics[kk] == 'curvature':
                metrichold[:,jj,ii,kk] = nib.load(f'/redacted/path/hippunfold/hippunfold/sub-{participantid[ii]}/surf/sub-{participantid[ii]}_hemi-{hemi[jj]}_space-T1w_den-0p5mm_label-hipp_{metrics[kk]}.shape.gii').agg_data()
            elif metrics[kk] == 'ODI' or metrics[kk] == 'wic':
                metrichold[:,jj,ii,kk] = nib.load(f'/redacted/path/NODDI/sub-{participantid[ii]}/midsurf/sub-{participantid[ii]}_hemi-{hemi[jj]}_space-T1w_den-0p5mm_label-hipp_midthickness_desc-{metrics[kk]}.shape.gii').agg_data()            
            elif metrics[kk] == 'b500_1200_MD':
                metrichold[:,jj,ii,kk] = nib.load(f'/redacted/path/DTI/sub-{participantid[ii]}/DTI_b500_1200/midsurf/sub-{participantid[ii]}_hemi-{hemi[jj]}_space-T1w_den-0p5mm_label-hipp_midthickness_desc-{metrics[kk]}.shape.gii').agg_data()         
            elif metrics[kk] == 'AP' or metrics[kk] == 'PD' or metrics[kk] == 'IO':
                metrichold[:,jj,ii,kk] = nib.load(f'/redacted/path/mrtrix/sub-{participantid[ii]}/cossim_peak1/sub-{participantid[ii]}_dir-{metrics[kk]}_hemi-{hemi[jj]}_space-T1w_label-hipp_desc-mrtrix_cossim_with_rotation.shape.gii').agg_data()
            else: 
                metrichold[:,jj,ii,kk] = nib.load(f'/redacted/path/SANDI/sub-{participantid[ii]}/midsurf/sub-{participantid[ii]}_hemi-{hemi[jj]}_space-T1w_den-0p5mm_label-hipp_midthickness_desc-{metrics[kk]}.shape.gii').agg_data()

### Hemisphere average

In [13]:
metrichold = np.mean(metrichold[:,:,:,:],axis=1)

In [ ]:
df = {}

df['age'] = age
df['participantid'] = participantid
df['sex'] = sex

dfcomb = pd.DataFrame(data=df)

### For each metric at each vertex, save the t-statistic with age

In [ ]:
from statsmodels.stats.anova import anova_lm
import statsmodels.formula.api as smf

t_stat = np.empty((metrichold.shape[0],len(metrics)))

for ii in range(metrichold.shape[0]):
    for jj in range(len(metrics)):
        
        dfcomb[f'{metrics[jj]}'] = metrichold[ii,:,jj].flatten()

        lm1 = smf.ols(formula=f'{metrics[jj]} ~ age + C(sex) + C(sex):age', data=dfcomb).fit()
        t_stat[ii,jj] = lm1.tvalues['age']

### Saving t-stat maps as giftis

In [ ]:
for ii in range(len(metrics)):
    gifticdata = nib.gifti.gifti.GiftiDataArray(t_stat[:,ii],intent='label')
    gifticdata = nib.gifti.gifti.GiftiImage(darrays=[gifticdata])
    nib.save(gifticdata, f'/redacted/path/manual_t-stat/{metrics[ii]}_hemi-combined_model-age-sex-interact_contrast-age_desc-tvalue.shape.gii')